# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
#библиотеки
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState
import warnings

from scipy import stats

In [2]:
#загружаем данные геологоразведки
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
#оценим информацию о данных
data_0.info(), data_1.info(), data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

(None, None, None)

In [4]:
#посмотрим на датасеты
display(data_0.head(), data_1.head(), data_2.head())

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [5]:
#учитывая описание данных, столбец id  неинформативен и будет лишним в обучении моделей, удалим его
data_0 = data_0.drop(['id'], axis=1)
data_1 = data_1.drop(['id'], axis=1)
data_2 = data_2.drop(['id'], axis=1)

In [6]:
#проверим на явные дубликаты
(data_0.duplicated().sum(), data_1.duplicated().sum(), data_2.duplicated().sum())

(0, 0, 0)

In [7]:
#проверим на пропуски
data_0.isna().sum(), data_1.isna().sum(), data_2.isna().sum()

(f0         0
 f1         0
 f2         0
 product    0
 dtype: int64,
 f0         0
 f1         0
 f2         0
 product    0
 dtype: int64,
 f0         0
 f1         0
 f2         0
 product    0
 dtype: int64)

In [8]:
#оценим распределение данных
display(data_0.describe(), data_1.describe(), data_2.describe())

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


In [9]:
#оценим зависимость признаков
data_0.corr().style.background_gradient(cmap='coolwarm')

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


In [10]:
data_1.corr().style.background_gradient(cmap='coolwarm')

,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


In [11]:
data_2.corr().style.background_gradient(cmap='coolwarm')

,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


Матрицы корреляции показывают на среднюю зависимость показателя объёма запасов product и признака f2 в первом и третьем датасете и сильную (0.99) во втором датасете. Интересно.

In [12]:
#посмотрим на уникальные значения data_1
data_1.nunique()

f0         100000
f1         100000
f2         100000
product        12
dtype: int64

Вывод. 
Данные загружены и обработаны. Пропусков и явных дубликатов нет. Столбец id удален ввиду не информативности. В результате по каждому региону датасет содержит 3 (три) значимых признака - f0, f1, f2 и 1 (один) целевой признак - product. Распределение данных в целом нормальное, о признаках нам ничего неизвестно, сформированы они алгоритмом и являются синтетическими. Матрицы корреляции показывают на среднюю зависимость показателя объёма запасов product от признака f2 в первом и третьем датасете и сильную (0.99) во втором датасете. Причина предварительно понятна - уникальных значений в колонке product всего 12, а столбец f2 имеет дисперсию значений от 0 до 5. 

## Обучение и проверка модели

In [13]:
# Метод обучения модели и прогнозирования
def model_lr(data):
    # зададим целевой признак и подготовим данные
    target = data['product']
    features = data.drop(['product'], axis=1)
    # Разбьём данные на обучающую и валидационную выборки в соотношении 75:25
    features_train, features_valid, target_train,target_valid = \
    train_test_split(features, target, test_size=0.25, random_state=12345)
    # инициируем модели линейной регрессии
    model = LinearRegression()
    # обучим модель
    model.fit(features_train, target_train)
    # сделаем прогнозы
    predicted_valid = model.predict(features_valid)
    # посчитаем rmse
    rmse = mean_squared_error(target_valid, predicted_valid)**0.5
    # посчитаем средний запас сырья
    predicted_valid_mean = predicted_valid.sum()/len(predicted_valid)
    return target_valid, predicted_valid, rmse, predicted_valid_mean

In [14]:
#основные показатели для каждого региона
target_valid_0, predicted_valid_0, rmse_0, predicted_valid_mean_0 = model_lr(data_0)
target_valid_1, predicted_valid_1, rmse_1, predicted_valid_mean_1 = model_lr(data_1)
target_valid_2, predicted_valid_2, rmse_2, predicted_valid_mean_2 = model_lr(data_2)

In [15]:
print('Регион_0. Средний запас сырья:', predicted_valid_mean_0, 'rmse:', rmse_0)
print('Регион_1. Средний запас сырья:', predicted_valid_mean_1, 'rmse:', rmse_1)
print('Регион_2. Средний запас сырья:', predicted_valid_mean_2, 'rmse:', rmse_2)

Регион_0. Средний запас сырья: 92.59256778438035 rmse: 37.5794217150813
Регион_1. Средний запас сырья: 68.728546895446 rmse: 0.893099286775617
Регион_2. Средний запас сырья: 94.96504596800489 rmse: 40.02970873393434


Вывод. Модели обучены и проверены на качество предсказания метрикой RMSE, показателем среднего расстояния между прогнозируемыми значениями модели и фактическими значениями в наборе данных. Чем ниже RMSE, тем лучше модель может соответствовать набору данных. В результате предказанные средние запасы сырья по трем регионам примерно соответствуют средним фактическим, но, учитывая значения RMSE лучше себя показала модель для второго региона, на втором месте первый регион. Третий регион с самой большой RMSE на последнем месте. Принимая во внимание, что фактические средние запасы сырья во втором регионе сравнительно меньше запасов первого и третьего регионов, то для окончательных рекомендаций по выбору скважин необходимо рассчитать возможную прибыль и оценить риски.

## Подготовка к расчёту прибыли

In [16]:
#сохраним ключевые значения для расчётов в отдельных переменных
total_points = 500
best_dev=200
budget = 10000000000
income_per_unit = 450000
income_per_barrel = 450
threshold_percent = 2.5

#рассчитаем средний объём сырья, достаточный для безубыточной разработки (тыс. баррелей)
budget_per_well = budget / best_dev
min_volume = budget_per_well / income_per_unit
print('Средний достаточный запас сырья:', round(min_volume, 3))

Средний достаточный запас сырья: 111.111


In [17]:
#сравним с предсказанным средним
print('Регион_0. Средний запас сырья:', predicted_valid_mean_0)
print('Регион_1. Средний запас сырья:', predicted_valid_mean_1)
print('Регион_2. Средний запас сырья:', predicted_valid_mean_2)

Регион_0. Средний запас сырья: 92.59256778438035
Регион_1. Средний запас сырья: 68.728546895446
Регион_2. Средний запас сырья: 94.96504596800489


In [18]:
#отклонение от среднего в Регион_0 (тыс. баррелей)
predicted_valid_mean_0 - min_volume

-18.518543326730764

In [19]:
#отклонение от среднего в Pегион_1 (тыс. баррелей)
predicted_valid_mean_1 - min_volume

-42.38256421566511

In [20]:
#отклонение от среднего в Pегион_2 (тыс. баррелей)
predicted_valid_mean_2 - min_volume

-16.146065143106227

Средний объём сырья, достаточный для безубыточной разработки, превышает предсказанные средние запасы в каждом из регионов, в этой связи нет возможности выделить лучший регион для возможной разработки новых скважин. Самое высокое отклонение в Регионе_1 (-42.38), далее Регион_0 (-18.52) и Регион_2 (-16.15)

## Расчёт прибыли и рисков 

In [21]:
#приведем целевые и прогнозные значения к одному типу, напишем функцию для Series
def series_type(target, pred):
    target = target.reset_index(drop=True)
    pred = pd.Series(pred)
    return target, pred

In [22]:
#приведем целевые и прогнозные значения по всем регионам к одному типу
target_valid_0, predicted_valid_0 = series_type(target_valid_0, predicted_valid_0)
target_valid_1, predicted_valid_1 = series_type(target_valid_1, predicted_valid_1)
target_valid_2, predicted_valid_2 = series_type(target_valid_2, predicted_valid_2)

In [23]:
#функция расчета прибыли (фактическая выручка 200 лучших скважин, минус инвестиции)
def total_income(target, probabilities, count, income_per_unit, budget):
    prob_sort = probabilities.sort_values(ascending=False)
    selected = target[prob_sort.index][:count]
    return int(income_per_unit * selected.sum() - budget)

In [24]:
#применим Bootstrap с 1000 выборок, чтобы найти распределение прибыли
def bootstrap_regions(target, predicted, best_dev, income_per_unit, budget):
    state = RandomState(12345)
    values = []
    for i in range(1000):
        target_subsample = target.sample(n = total_points, replace=True, random_state=state)
        pred_subsumple = predicted[target_subsample.index]
        values.append(total_income(target_subsample, pred_subsumple,\
                                   best_dev, income_per_unit, budget))
    values = pd.Series(values)
    values_mean = int(values.mean())
    lower = int(values.quantile(q=0.025))
    upper = int(values.quantile(q=0.975))
    risk = int(len(values[values < 0]) / len(values) * 100)
    return values_mean, lower, upper, risk

In [25]:
#рассчитаем основные параметры для всех регионов
values_mean_0, lower_0, upper_0, risk_0  = bootstrap_regions(target_valid_0, predicted_valid_0,\
                                                             best_dev, income_per_unit, budget)
values_mean_1, lower_1, upper_1, risk_1  = bootstrap_regions(target_valid_1, predicted_valid_1, \
                                                             best_dev, income_per_unit, budget)
values_mean_2, lower_2, upper_2, risk_2  = bootstrap_regions(target_valid_2, predicted_valid_2, \
                                                             best_dev, income_per_unit, budget)

In [26]:
#оценим среднюю прибыль 200 лучших месторождений
print('Средняя прибыль лучших месторождений Регион_0', values_mean_0, 'руб')
print('Средняя прибыль лучших месторождений Регион_1', values_mean_1, 'руб')
print('Средняя прибыль лучших месторождений Регион_2', values_mean_2, 'руб')

Средняя прибыль лучших месторождений Регион_0 425938526 руб
Средняя прибыль лучших месторождений Регион_1 515222772 руб
Средняя прибыль лучших месторождений Регион_2 435008362 руб


In [27]:
#выведем 95% доверительный интервал для средней прибыли 200 лучших месторождений
print('95% доверительный интервал Регион_0', lower_0, upper_0)
print('95% доверительный интервал Регион_1', lower_1, upper_1)
print('95% доверительный интервал Регион_2', lower_2, upper_2)

95% доверительный интервал Регион_0 -102090093 947976352
95% доверительный интервал Регион_1 68873225 931547590
95% доверительный интервал Регион_2 -128880546 969706953


In [28]:
#оценим риск убытков при добыче сырья в данных регионах
print('Риск убытков Регион_0', risk_0, '%')
print('Риск убытков Регион_1', risk_1, '%')
print('Риск убытков Регион_2', risk_2, '%')

Риск убытков Регион_0 6 %
Риск убытков Регион_1 1 %
Риск убытков Регион_2 6 %


Вывод. Согласно распределению среднего значения прибыли, лидером оказался Регион_1, средняя прибыль в этом регионе составит 515.22 млн.рублей. Два других региона не прошли отбор по минимально допустимому порогу убытка в 2,5%. Таким образом несмотря на сравнительно меньшие запасы сырья, по сравнению с другими регионами, средняя доходность со скважин в Регион_1 прогнозируется выше других.

Общий вывод. Мы исследовали данные по запасам сырья в скважинах трёх регионов. В целом данные были хорошо подготовлены - из значимых изменений удаление колонок с id скважины. 
Было установлено, что средний запас ресурсов по регионам недостаточен даже для окупаемости вложений на разработку. Мы обучили модель и выполнили предсказания при помощи линейной регрессии, а так же применили технологию bootstrap с разделением каждого предсказанного значения прибыли по регионам на 1000 выборок.В результате мы определили доверительный интревал получения прибыли в 95%, ограничив вероятность убытка величиной менее 2,5%. На основе этих данных смогли выбрать наиболее перспективный регион для разработки.
Таким образом, несмотря на сравнительно меньшие предсказания запасов сырья в по сравнению с другими регионами, но со средней прибылью в 515.22 млн.руб. и наименьшим риском убытков в 1%  Заказчику рекомендован для разработки скважин второй регион (в проекте Регион_1).